# Four tank system example - Fault Information System, column resoning

In [1]:
import pandas as pd
from isolability import pairwise_relationships as pr
from isolability import martices_and_indices as ind
from four_tanks.es import ess

#### Fault Information System

In [2]:
fis_df = pd.read_csv('four_tanks/fis.csv', sep=';', index_col=0, dtype=str)
fis_df

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10
s_1,-1,-1,-1,-1,+1,+1,+1,"-1,+1","-1,+1",0
s_2,-1,-1,-1,-1,-1,-1,-1,"-1,+1",0,"-1,+1"
s_3,0,-1,-1,-1,-1,-1,-1,0,"-1,+1","-1,+1"


In [3]:
fis_df = fis_df.map(lambda x: set(a.strip() for a in x.split(',')))
fis_df

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10
s_1,{-1},{-1},{-1},{-1},{+1},{+1},{+1},"{-1, +1}","{-1, +1}",{0}
s_2,{-1},{-1},{-1},{-1},{-1},{-1},{-1},"{-1, +1}",{0},"{-1, +1}"
s_3,{0},{-1},{-1},{-1},{-1},{-1},{-1},{0},"{-1, +1}","{-1, +1}"


#### Exploration of pairwise isolabity relationships
* Function arguments are fault **indices** starting from 0, so pr.RS_C_FIS(fis_df, 1, 2) checks faults $f_2$ and $f_3$
* Avaliable relations:
    - RN_C_FIS - non-isolable
    - RUN_C_FIS - unilaterally non-isolable
    - RUC_C_FIS - unilaterally conditionally isolable
    - RC_C_FIS - bilaterally conditionally isolable
    - RW_C_FIS - weakly isolable
    - RS_C_FIS - strongly isolable

In [4]:
pr.RS_C_FIS(fis_df, 7, 8)

True

#### EIM matrix

In [5]:
eim = ind.EIM_C_FIS(fis_df)
eim

array([[0. , 1. , 1. , 1. , 2. , 2. , 2. , 0. , 2. , 2. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [2. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [2. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [2. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [0.4, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 2. , 2. ],
       [2. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 0. , 2. ],
       [2. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 2. , 0. ]])

#### $C\Delta$ coefficient

In [6]:
c_delta = ind.C_Delta(eim)
print(f"{c_delta:.3f}")

1.027


#### $C^*\Delta$ coefficient

In [7]:
c_star_delta = ind.C_star_Delta(eim)
print(f"{c_star_delta:.3f}")

0.849


#### IMES matrix

In [8]:
imes = ind.IMES(ess)
imes

array([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

#### Weights for composition of EIM and IMES

In [9]:
comp_weights = pd.read_csv('conf/IMES_weights_FIS_C.csv', sep=';', index_col=0)
comp_weights

,RW_ES,RJW_ES,RN_ES
RS_Cn_FIS,2.0,2.0,2.0
RW_C_FIS,2.0,1.5,1.0
RC_C_FIS,1.7,1.2,0.7
RUC_C_FIS,1.4,0.7,0.4
RUN_C_FIS,1.0,0.5,0.0
RN_C_FIS,1.0,0.5,0.0


#### EIM$^*$

In [10]:
eim_star = ind.EIM_star_C_FIS(fis_df, ess, comp_weights)
eim_star

array([[0. , 1. , 2. , 2. , 2. , 2. , 2. , 0. , 2. , 2. ],
       [1. , 0. , 1. , 1. , 1. , 1. , 2. , 1. , 1. , 1. ],
       [2. , 1. , 0. , 0. , 2. , 1. , 1. , 1. , 1. , 1. ],
       [2. , 1. , 0. , 0. , 2. , 1. , 1. , 1. , 1. , 1. ],
       [2. , 1. , 2. , 2. , 0. , 0. , 1. , 1. , 1. , 1. ],
       [2. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [2. , 2. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 1. ],
       [0.4, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 2. , 2. ],
       [2. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 0. , 2. ],
       [2. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 2. , 0. ]])

#### $C \Delta (EIM^*)$

In [11]:
c_delta = ind.C_Delta(eim_star)
print(f"{c_delta:.3f}")

1.204


#### $C^* \Delta (EIM^*)$

In [12]:
c_star_delta = ind.C_star_Delta(eim_star)
print(f"{c_star_delta:.3f}")

0.916
